In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
data = pd.read_csv("breast-cancer-wisconsin.csv", encoding = 'utf-8')

In [3]:
data.head()

,code,Clump_Thickness,Cell_Size,Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,0
1,1002945,5,4,4,5,7,10,3,2,1,0
2,1015425,3,1,1,1,2,2,3,1,1,0
3,1016277,6,8,8,1,3,4,3,7,1,0
4,1017023,4,1,1,3,2,1,3,1,1,0


In [4]:
X = data[data.columns[1:10]]
Y = data[['Class']]

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify = Y, random_state = 42)

In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

## 투표기반 앙상블

**분류**

강한 학습기(범주)

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

logit_model = LogisticRegression(random_state = 42)
rnf_model = RandomForestClassifier(random_state = 42)
svm_model = SVC(random_state = 42)

voting_hard = VotingClassifier(estimators = [('lr', logit_model), ('rf', rnf_model), ('svc', svm_model)], voting = "hard")
voting_hard.fit(X_scaled_train, Y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

In [9]:
from sklearn.metrics import accuracy_score

for clf in (logit_model, rnf_model, svm_model, voting_hard) : 
    clf.fit(X_scaled_train, Y_train)
    Y_pred = clf.predict(X_scaled_test)
    print(clf.__class__.__name__, accuracy_score(Y_test, Y_pred))

LogisticRegression 0.9590643274853801
RandomForestClassifier 0.9649122807017544
SVC 0.9649122807017544
VotingClassifier 0.9649122807017544


In [10]:
from sklearn.metrics import confusion_matrix
log_pred_train = logit_model.predict(X_scaled_train)
log_confusion_train = confusion_matrix(Y_train, log_pred_train)
print("로지스틱 분류기 훈련데이터 오차행렬 : \n", log_confusion_train)

log_pred_test = logit_model.predict(X_scaled_test)
log_confusion_test = confusion_matrix(Y_test, log_pred_test)
print("로지스틱 분류기 테스트데이터 오차행렬 : \n", log_confusion_test)

로지스틱 분류기 훈련데이터 오차행렬 : 
 [[328   5]
 [  9 170]]
로지스틱 분류기 테스트데이터 오차행렬 : 
 [[106   5]
 [  2  58]]


In [12]:
svm_pred_train = svm_model.predict(X_scaled_train)
svm_confusion_train = confusion_matrix(Y_train, svm_pred_train)
print("SVM 훈련데이터 오차행렬 : \n", svm_confusion_train)

svm_pred_test = svm_model.predict(X_scaled_test)
svm_confusion_test = confusion_matrix(Y_test, svm_pred_test)
print("svm 테스트데이터 오차행렬 : \n", svm_confusion_test)

SVM 훈련데이터 오차행렬 : 
 [[329   4]
 [  4 175]]
svm 테스트데이터 오차행렬 : 
 [[106   5]
 [  1  59]]


In [14]:
rnf_pred_train = rnf_model.predict(X_scaled_train)
rnf_confusion_train = confusion_matrix(Y_train, rnf_pred_train)
print("랜덤포레스트 훈련데이터 오차행렬 : \n", rnf_confusion_train)

rnf_pred_test = rnf_model.predict(X_scaled_test)
rnf_confusion_test = confusion_matrix(Y_test, rnf_pred_test)
print("랜덤포레스트 테스트데이터 오차행렬 : \n", rnf_confusion_test)

랜덤포레스트 훈련데이터 오차행렬 : 
 [[333   0]
 [  0 179]]
랜덤포레스트 테스트데이터 오차행렬 : 
 [[106   5]
 [  1  59]]


In [16]:
voting_pred_train = voting_hard.predict(X_scaled_train)
voting_confusion_train = confusion_matrix(Y_train, voting_pred_train)
print("투표분류기 훈련데이터 오차행렬 : \n", voting_confusion_train)

voting_pred_test = voting_hard.predict(X_scaled_test)
voting_confusion_test = confusion_matrix(Y_test, voting_pred_test)
print("투표분류기 테스트데이터 오차행렬 : \n", voting_confusion_test)

투표분류기 훈련데이터 오차행렬 : 
 [[329   4]
 [  4 175]]
투표분류기 테스트데이터 오차행렬 : 
 [[106   5]
 [  1  59]]


약한 학습기(확률)

In [28]:
voting_soft = VotingClassifier(estimators = [('lr', logit_model), ('rf', rnf_model), ('svc', svm_model)], voting = 'soft')
voting_soft.fit(X_scaled_train, Y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))],
                 voting='soft')

In [31]:
from sklearn.metrics import accuracy_score

for clf in (logit_model, rnf_model, svm_model, voting_soft) : 
    clf.fit(X_scaled_train, Y_train)
    Y_pred = clf.predict(X_scaled_test)
    print(clf.__class__.__name__, accuracy_score(Y_test, Y_pred))

LogisticRegression 0.9590643274853801
RandomForestClassifier 0.9649122807017544
SVC 0.9649122807017544


AttributeError: predict_proba is not available when  probability=False

In [32]:
voting_pred_train = voting_soft.predict(X_scaled_train)
voting_confusion_train = confusion_matrix(Y_train, voting_pred_train)
print("투표분류기 훈련데이터 오차행렬 : \n", voting_confusion_train)

voting_pred_test = voting_soft.predict(X_scaled_test)
voting_confusion_test = confusion_matrix(Y_test, voting_pred_test)
print("투표분류기 훈련데이터 오차행렬 : \n", voting_confusion_test)

AttributeError: predict_proba is not available when  probability=False

**회귀**

In [7]:
data2 = pd.read_csv('house_price.csv', encoding = 'utf-8')
data2.head()

,housing_age,income,bedrooms,households,rooms,house_value
0,23,6.7770,0.141112,2.442244,8.103960,500000
1,49,6.0199,0.160984,2.726688,5.752412,500000
2,35,5.1155,0.249061,1.902676,3.888078,500000
3,32,4.7109,0.231383,1.913669,4.508393,500000
4,21,4.5625,0.255583,3.092664,4.667954,500000


In [8]:
x = data2[data2.columns[1:5]]
y = data2[['house_value']]

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 42)

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x_train)
x_scaled_train = scaler.transform(x_train)
x_scaled_test = scaler.transform(x_test)

In [37]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor

linear_model = LinearRegression()
rnf_model = RandomForestRegressor(random_state = 42)

voting_regressor = VotingRegressor(estimators = [('lr', linear_model), ('rf', rnf_model)])
voting_regressor.fit(x_scaled_train, y_train)

VotingRegressor(estimators=[('lr', LinearRegression()),
                            ('rf', RandomForestRegressor(random_state=42))])

In [38]:
pred_train = voting_regressor.predict(x_scaled_train)
voting_regressor.score(x_scaled_train, y_train)

0.7962532705428835

In [39]:
pred_test = voting_regressor.predict(x_scaled_test)
voting_regressor.score(x_scaled_test, y_test)

0.5936371957936409

In [41]:
import numpy as np
from sklearn.metrics import mean_squared_error
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)
print("훈련데이터 RMSE : ", np.sqrt(MSE_train))
print("평가데이터 RMSE : ", np.sqrt(MSE_test))

훈련데이터 RMSE :  43082.050654857834
평가데이터 RMSE :  60942.38524353489


## 배깅

**분류**

In [11]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
model = BaggingClassifier(base_estimator = SVC(), n_estimators = 10, random_state = 0)
model.fit(X_scaled_train, Y_train)
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, Y_train)

0.982421875

In [12]:
from sklearn.metrics import confusion_matrix
confusion_train = confusion_matrix(Y_train, pred_train)
print("훈련데이터 오차행렬 : \n", confusion_train)

훈련데이터 오차행렬 : 
 [[329   4]
 [  5 174]]


In [13]:
from sklearn.metrics import classification_report
cfreport_train = classification_report(Y_train, pred_train)
print("분류예측 레포트 : \n", cfreport_train)

분류예측 레포트 : 
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       333
           1       0.98      0.97      0.97       179

    accuracy                           0.98       512
   macro avg       0.98      0.98      0.98       512
weighted avg       0.98      0.98      0.98       512



In [14]:
pred_test = model.predict(X_scaled_test)
model.score(X_scaled_test, Y_test)

0.9590643274853801

In [15]:
confusion_test = confusion_matrix(Y_test, pred_test)
print("테스트데이터 오차행렬 : \n", confusion_test)

테스트데이터 오차행렬 : 
 [[106   5]
 [  2  58]]


In [16]:
cfreport_test = classification_report(Y_test, pred_test)
print("분류예측 레포트 : \n", cfreport_test)

분류예측 레포트 : 
               precision    recall  f1-score   support

           0       0.98      0.95      0.97       111
           1       0.92      0.97      0.94        60

    accuracy                           0.96       171
   macro avg       0.95      0.96      0.96       171
weighted avg       0.96      0.96      0.96       171



**회귀**

In [17]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor
model = BaggingRegressor(base_estimator = KNeighborsRegressor(), n_estimators = 10, random_state = 0)
model.fit(x_scaled_train, y_train)
pred_train = model.predict(x_scaled_train)
model.score(x_scaled_train, y_train)

0.6928982134381334

In [18]:
pred_test = model.predict(x_scaled_test)
model.score(x_scaled_test, y_test)

0.5612676280708411

In [19]:
import numpy as np
from sklearn.metrics import mean_squared_error
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)
print("훈련 데이터 RMSE : ", np.sqrt(MSE_train))
print("평가 데이터 RMSE : ", np.sqrt(MSE_test))

훈련 데이터 RMSE :  52892.27111989147
평가 데이터 RMSE :  63323.12131927774


## 앙상블 부스팅

**분류**

AdaBoosting

In [20]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators = 100, random_state = 0)
model.fit(X_scaled_train, Y_train)
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, Y_train)

1.0

In [21]:
confusion_train = confusion_matrix(Y_train, pred_train)
print("훈련데이터 오차행렬 : \n", confusion_train)

훈련데이터 오차행렬 : 
 [[333   0]
 [  0 179]]


In [22]:
cfreport_train = classification_report(Y_train, pred_train)
print("분류예측 레포트 : \n", cfreport_train)

분류예측 레포트 : 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       333
           1       1.00      1.00      1.00       179

    accuracy                           1.00       512
   macro avg       1.00      1.00      1.00       512
weighted avg       1.00      1.00      1.00       512



In [23]:
pred_test = model.predict(X_scaled_test)
model.score(X_scaled_test, Y_test)

0.9532163742690059

In [24]:
confusion_test = confusion_matrix(Y_test, pred_test)
print("테스트데이터 오차행렬 : \n", confusion_test)

테스트데이터 오차행렬 : 
 [[106   5]
 [  3  57]]


In [25]:
cfreport_test = classification_report(Y_test, pred_test)
print("분류예측 레포트 : \n", cfreport_test)

분류예측 레포트 : 
               precision    recall  f1-score   support

           0       0.97      0.95      0.96       111
           1       0.92      0.95      0.93        60

    accuracy                           0.95       171
   macro avg       0.95      0.95      0.95       171
weighted avg       0.95      0.95      0.95       171



GradientBoosting

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(n_estimators = 100, learning_rate = 0.1, max_depth = 1, random_state = 0)
model.fit(X_scaled_train, Y_train)
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, Y_train)

0.984375

In [27]:
confusion_train = confusion_matrix(Y_train, pred_train)
print("훈련데이터 오차행렬 : \n", confusion_train)

훈련데이터 오차행렬 : 
 [[329   4]
 [  4 175]]


In [28]:
pred_test = model.predict(X_scaled_test)
model.score(X_scaled_test, Y_test)

0.9649122807017544

In [29]:
confusion_test = confusion_matrix(Y_test, pred_test)
print("평가데이터 오차행렬 : \n", confusion_test)

평가데이터 오차행렬 : 
 [[106   5]
 [  1  59]]


**회귀**

AdaBoosting

In [31]:
from sklearn.ensemble import AdaBoostRegressor
model = AdaBoostRegressor(random_state = 0, n_estimators = 100)
model.fit(x_scaled_train, y_train)
pred_train = model.predict(x_scaled_train)
model.score(x_scaled_train, y_train)

0.4353130085971758

In [32]:
pred_test = model.predict(x_scaled_test)
model.score(x_scaled_test, y_test)

0.43568387094087124

In [33]:
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)
print("훈련데이터 RMSE : ",np.sqrt(MSE_train))
print("평가데이터 RMSE : ",np.sqrt(MSE_test))

훈련데이터 RMSE :  71722.42012035428
평가데이터 RMSE :  71816.41231019037


GradientBoosting

In [35]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(random_state = 0)
model.fit(x_scaled_train, y_train)
pred_train = model.predict(x_scaled_train)
model.score(x_scaled_train, y_train)

0.6178724780500952

In [36]:
pred_test = model.predict(x_scaled_test)
model.score(x_scaled_test, y_test)

0.5974112241813845

In [37]:
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)
print("훈련데이터 RMSE : ", np.sqrt(MSE_train))
print("평가데이터 RMSE : ", np.sqrt(MSE_test))

훈련데이터 RMSE :  59000.433545962376
평가데이터 RMSE :  60658.72886338227


## 앙상블 스태킹

**분류**

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
estimators = [('rf', RandomForestClassifier(n_estimators = 10, random_state = 42)), ('svc', SVC(random_state = 0))]
model = StackingClassifier(estimators = estimators, final_estimator = LogisticRegression())
model.fit(X_scaled_train, Y_train)
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, Y_train)

0.986328125

In [12]:
from sklearn.metrics import confusion_matrix
confusion_train = confusion_matrix(Y_train, pred_train)
print("훈련데이터 오차행렬 : \n", confusion_train)

훈련데이터 오차행렬 : 
 [[330   3]
 [  4 175]]


In [13]:
from sklearn.metrics import classification_report
cfreport_train = classification_report(Y_train, pred_train)
print("분류예측 레포트 : \n", cfreport_train)

분류예측 레포트 : 
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       333
           1       0.98      0.98      0.98       179

    accuracy                           0.99       512
   macro avg       0.99      0.98      0.98       512
weighted avg       0.99      0.99      0.99       512



In [14]:
pred_test = model.predict(X_scaled_test)
model.score(X_scaled_test, Y_test)

0.9649122807017544

In [15]:
confusion_test = confusion_matrix(Y_test, pred_test)
print("평가데이터 오차행렬 : \n", confusion_test)

평가데이터 오차행렬 : 
 [[106   5]
 [  1  59]]


In [16]:
cfreport_test = classification_report(Y_test, pred_test)
print("분류예측 레포트 : \n", cfreport_test)

분류예측 레포트 : 
               precision    recall  f1-score   support

           0       0.99      0.95      0.97       111
           1       0.92      0.98      0.95        60

    accuracy                           0.96       171
   macro avg       0.96      0.97      0.96       171
weighted avg       0.97      0.96      0.97       171



**회귀**

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
estimators = [('lr', LinearRegression()), ('knn', KNeighborsRegressor())]
model = StackingRegressor(estimators = estimators, final_estimator = RandomForestRegressor(n_estimators = 10, random_state = 42))
model.fit(x_scaled_train, y_train)
pred_train = model.predict(x_scaled_train)
model.score(x_scaled_train, y_train)

0.543404932348004

In [20]:
pred_test = model.predict(x_scaled_test)
model.score(x_scaled_test, y_test)

0.4781188528801523

In [23]:
import numpy as np
from sklearn.metrics import mean_squared_error
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)
print("훈련 데이터 RMSE : ", np.sqrt(MSE_train))
print("평가 데이터 RMSE : ", np.sqrt(MSE_test))

훈련 데이터 RMSE :  64493.60476580374
평가 데이터 RMSE :  69063.45138802647
